can also check the linear independence of rho and Jn here as well!

# Imports:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression

from scipy.stats import chi2
from collections import Counter

from itertools import permutations
from tqdm import tqdm, trange

In [2]:
import matplotlib.pyplot as plt

import pickle

import statsmodels.api as sm

from collections import defaultdict

from scipy.linalg import sqrtm

from scipy.special import expit, logit

# Helper Functions:

In [1]:
from scipy.stats import norm

ModuleNotFoundError: No module named 'scipy'

In [ ]:
import sys

sys.path.append('../')

from ddc_utils import *
from data_generating_utils import *

# Hyperparams:

In [7]:
pop_index = 1
iter_val = 0

In [8]:
rand_generator = np.random.default_rng(seed=333 * pop_index + iter_val)

In [9]:
population_size = 100_000

number_of_coefficients = 1

num_iters_per_population = 25_000
true_beta = 0.610
link = 'Probit'

In [10]:
ALL_SAMPLE_SIZES = [1000]

# Run:

In [11]:
feature_cols = [f'x_{i}' for i in range(number_of_coefficients)]

In [12]:
# hypervariabes where things will be saved as key of sample size -> list.
all_jns_per_sample_size_biased = defaultdict(list)
all_ddc_per_sample_size_biased = defaultdict(list)
all_sample_beta_per_sample_size_biased = defaultdict(list)

all_jns_per_sample_size_full = defaultdict(list)
all_ddc_per_sample_size_full = defaultdict(list)
all_sample_beta_per_sample_size_full = defaultdict(list)



all_realized_sample_sizes_per_sample_size = defaultdict(list)
all_pop_beta_per_sample_size = defaultdict(list)

sample_specific_non_separable_count = {}

In [13]:
sample_probability_centering = 0.77
sample_probability_bias_factor = 1

In [14]:
pop_data = generate_population_data(population_size, number_of_coefficients, 
                                    feature_cols = feature_cols, true_beta = true_beta, link = link)

In [15]:
pickle_filename = f'base_population_data_{link}_{true_beta}.pickle'
with open(pickle_filename, 'wb') as handle:
    pickle.dump(pop_data, handle)

In [16]:
model_iteration = [(0, 'Logit'), (1, 'Probit'), (2, 'CLogLog')]

#### get population-level statistics:

In [17]:
pop_x = pop_data[feature_cols]
pop_y = pop_data['y']

pop_model = sm.Logit(endog = pop_y, exog = pop_x).fit(disp=0)
pop_beta = np.array(pop_model.params)
pop_gs = pop_x * (np.array(pop_y).reshape((population_size, 1)) - \
              np.array(pop_model.predict()).reshape((population_size, 1)))

compute the population models:

In [18]:
def compute_all_three_logistic_models(x_data, y_data):
    temp_logit_model = sm.Logit(endog = y_data, exog = x_data).fit(disp=0)
    temp_probit_model = sm.GLM(y_data, x_data, family=sm.families.Binomial(link=sm.families.links.Probit())).fit()
    temp_cloglog_model = sm.GLM(y_data, x_data, family=sm.families.Binomial(link=sm.families.links.CLogLog())).fit()
    return [temp_logit_model, temp_probit_model, temp_cloglog_model]

In [19]:
population_models = compute_all_three_logistic_models(pop_x, pop_y)
pop_logit_model, pop_probit_model, pop_cloglog_model = population_models

save their betas:

In [20]:
pop_betas = [np.array(pop_logit_model.params), 
             np.array(pop_probit_model.params), 
             np.array(pop_cloglog_model.params)]

In [21]:
pop_betas

[array([1.01802156]), array([0.61579984]), array([0.5698061])]

compute the gs:

In [22]:
pop_gs = {}

In [23]:
pop_gs['Logit'] = pop_x * (np.array(pop_y).reshape((population_size, 1)) - \
                    np.array(pop_logit_model.predict()).reshape((population_size, 1)))

In [24]:
# compute probit gs 
temp_mu_is = pop_probit_model.predict()
variance_denominator = 1/((temp_mu_is) * (1 - temp_mu_is))
dmu_dg = norm.pdf(pop_x @ pop_probit_model.params)
pop_gs['Probit'] = pop_x.mul(((pop_y - temp_mu_is) * variance_denominator * dmu_dg), axis=0)

In [25]:
# compute cloglog gs 
temp_mu_is = pop_cloglog_model.predict()
variance_denominator = 1/((temp_mu_is) * (1 - temp_mu_is))
dmu_dg = np.log(1 - temp_mu_is) * (temp_mu_is - 1)
pop_gs['CLogLog'] = pop_x.mul(((pop_y - temp_mu_is) * variance_denominator * dmu_dg), axis=0)

#### actually run:

In [26]:
for temp_sample_size in tqdm(ALL_SAMPLE_SIZES):
    non_separable_count = 0
    num_iters_per_population = 10_000
        
    for _ in trange(num_iters_per_population, mininterval=10):
        # use sampling scheme to sample data:
        obtained_valid_sample = False
        
        while not obtained_valid_sample:
            pop_data['r0'] = 0
            pop_data.loc[np.random.choice(pop_data.index, size = temp_sample_size, replace=False), 'r0'] = 1

            pop_data['r'] = 0
            
            full_sampled_data = pop_data[pop_data['r0'] == 1]

            marginal_probabilities = expit(logit(sample_probability_centering) + \
                                           sample_probability_bias_factor * (2* full_sampled_data['y'] - 1) * \
                                           full_sampled_data['x_0'])

            other_sample_indices = marginal_probabilities.index[rand_generator.binomial(n=1, p = marginal_probabilities) == 1]

            pop_data.loc[other_sample_indices, 'r'] = 1
            # sample_data here means the biased sample data.
            sample_data = pop_data[pop_data['r'] == 1]
            realised_sample_size = len(other_sample_indices)
            
            if realised_sample_size < 1_000:
                separability_check_df = sample_data[['x_0', 'y']].groupby('y')['x_0'].agg(['min', 'max'])
                is_not_separable = (len(separability_check_df) < 2) or \
                                    (separability_check_df.iloc[0, 0] > separability_check_df.iloc[1, 1]) or \
                                    (separability_check_df.iloc[1, 0] > separability_check_df.iloc[0, 1])

                if is_not_separable:
                    non_separable_count = non_separable_count + 1
                    continue

            else:
                ...
            obtained_valid_sample = True
            
            # compute biased x, y, model, beta
            sample_x = sample_data[feature_cols]
            sample_y = sample_data['y']
            sample_models = compute_all_three_logistic_models(sample_x, sample_y)
            sample_betas = [sample_model.params for sample_model in sample_models]
            sample_r = pop_data['r']
        
        # compute full x, y, model, beta
        sample_x_full = full_sampled_data[feature_cols]
        sample_y_full = full_sampled_data['y']
        sample_models_full = compute_all_three_logistic_models(sample_x_full, sample_y_full)
        sample_betas_full = [sample_model_full.params for sample_model_full in sample_models_full]
        sample_r_full = pop_data['r0']
        
        
        
        # compute biased versions of things:
        all_sample_beta_per_sample_size_biased[temp_sample_size].append(
            [pd.Series(sample_beta) for sample_beta in sample_betas]
        )        
        all_ddc_per_sample_size_biased[temp_sample_size].append(
                        [pop_gs[model_type].corrwith(sample_r)[['x_0']] for _, model_type in model_iteration]
        )
        all_jns_per_sample_size_biased[temp_sample_size].append(
            [compute_average_jn(pop_betas[model_index], sample_betas[model_index], 
                               sample_x, sample_y, model_type = model_type) for model_index, model_type in model_iteration]
        )
            
        all_realized_sample_sizes_per_sample_size[temp_sample_size].append(realised_sample_size)
        
        # compute full versions of things:
        all_sample_beta_per_sample_size_full[temp_sample_size].append(
            [pd.Series(sample_beta) for sample_beta in sample_betas_full]
        )
        
        all_ddc_per_sample_size_full[temp_sample_size].append(
            [pop_gs[model_type].corrwith(sample_r_full)[['x_0']] for _, model_type in model_iteration]
        )
        
        all_jns_per_sample_size_full[temp_sample_size].append(
            [compute_average_jn(pop_betas[model_index], sample_betas_full[model_index], 
                               sample_x_full, sample_y_full, 
                               model_type = model_type) for model_index, model_type in model_iteration]
        )
        
    sample_specific_non_separable_count[temp_sample_size] = non_separable_count
    

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

  1%|▊                                                                             | 107/10000 [00:10<15:30, 10.63it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.c


 21%|████████████████▎                                                            | 2115/10000 [03:03<10:51, 12.10it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  r

 47%|████████████████████████████████████▍                                        | 4734/10000 [07:03<08:01, 10.93it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 50%|██████████████████████████████████████▎                                      | 4975/10000 [07:24<07:18, 11.47it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 51%|███████████████████████████████████████▏                                     | 5096/10000 [07:34<07:05, 11.54it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: 


 75%|█████████████████████████████████████████████████████████▍                   | 7452/10000 [11:22<04:13, 10.05it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 76%|██████████████████████████████████████████████████████████▏                  | 7555/10000 [11:35<04:28,  9.12it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 76%|██████████


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [15:41<00:00, 941.64s/it]


#### combine data for each sample size:

In [27]:
sample_specific_non_separable_count

{1000: 0}

In [28]:
all_data_per_ss = []

In [29]:
all_raw_data = [all_jns_per_sample_size_biased, all_ddc_per_sample_size_biased, all_sample_beta_per_sample_size_biased, \
                all_jns_per_sample_size_full, all_ddc_per_sample_size_full, all_sample_beta_per_sample_size_full, \
                all_realized_sample_sizes_per_sample_size, all_pop_beta_per_sample_size]

In [30]:
pickle_filename = f'all_raw_data_{link}_{true_beta}.pickle'
with open(pickle_filename, 'wb') as handle:
    pickle.dump(all_raw_data, handle)

In [31]:
pickle_filename = f'pop_betas_{link}_{true_beta}.pickle'
with open(pickle_filename, 'wb') as handle:
    pickle.dump(pop_betas, handle)